# Clustering Analysis v2 — Changelog

This notebook documents every change made in `clustering_analysis_v2.ipynb` relative to the baseline `clustering_analysis.ipynb`. The improvements fall into three categories: **reproducibility**, **methodological rigour**, and **code quality**.

---

## Summary of Changes

| # | Area | Baseline | v2 | Cells affected |
|---|------|----------|----|----------------|
| 1 | Reproducibility | No random seeds set | Global `SEED = 42` with `random.seed`, `np.random.seed` | cell-3 (imports) |
| 2 | Unused import | `from collections import Counter` imported but never used | Removed | cell-3 (imports) |
| 3 | UMAP performance | Two independent UMAP fits, each computing its own NN graph | NN graph computed once via `nearest_neighbors()`, shared via `precomputed_knn` | cell-11, cell-12 |
| 4 | Plot code | Inline scatter logic duplicated across 5+ cells | Reusable `umap_scatter()` helper function | cell-14, cell-15, cell-20, cell-21, cell-30 |
| 5 | HDBSCAN sweep | 1D sweep over `min_cluster_size` only (5 values) | 3D sweep over `min_cluster_size` x `min_samples` x `epsilon` (64 configs) | cell-17 |
| 6 | Selection metric | Best config chosen by **ARI** (uses ground-truth labels — circular) | Best config chosen by **DBCV** (internal, unsupervised) | cell-17, cell-18 |
| 7 | Cluster count | No constraint on number of clusters | Filtered to **<= 30 clusters** before DBCV selection | cell-17, cell-18 |
| 8 | DBCV computation | Not computed (`gen_min_span_tree` not enabled) | `gen_min_span_tree=True` on all HDBSCAN runs | cell-17, cell-18 |
| 9 | Noise handling | Noise points excluded from all evaluation | `approximate_predict` assigns noise to nearest cluster; both excl-noise and full-dataset metrics reported | cell-23 |
| 10 | Centroid distance | Euclidean distance for representative sample selection | **Cosine distance** (consistent with UMAP metric) | cell-32 |
| 11 | LLM API calls | Sequential (one-at-a-time) | **Concurrent** via `AsyncAnthropic` + `asyncio.gather` | cell-32 |

---

## Background: What Is DBCV?

**DBCV** (Density-Based Clustering Validation) is an **internal** cluster validity index designed specifically for density-based clustering algorithms like HDBSCAN. It was introduced by Moulavi et al. (2014) and is available in the HDBSCAN library as `clusterer.relative_validity_`.

### The Core Idea

Traditional internal metrics like the Silhouette Score assume clusters are convex, globular shapes — they measure how close each point is to the center of its own cluster versus the center of the nearest other cluster. This assumption breaks down for density-based algorithms, which can discover clusters of arbitrary shape.

DBCV instead evaluates clustering quality in terms of **density**:

1. **Intra-cluster density** — For each cluster, DBCV builds a *minimum spanning tree* (MST) over its member points using a **mutual reachability distance** (a density-aware distance that inflates distances in sparse regions). The *densest* internal connection within the cluster defines how tightly packed it is.

2. **Inter-cluster density** — For each pair of clusters, DBCV finds the densest path *between* them. If clusters are well-separated, this inter-cluster density will be low.

3. **Per-cluster score** — For each cluster, DBCV computes:

$$\text{DBCV}_i = \frac{\text{min inter-cluster density} - \text{max intra-cluster density}}{\max(\text{min inter-cluster density},\; \text{max intra-cluster density})}$$

The value ranges from **-1 to +1**: positive means the cluster is denser internally than the space separating it from other clusters; negative means clusters are overlapping.

4. **Global score** — The overall DBCV is the density-weighted average of per-cluster scores:

$$\text{DBCV} = \frac{\sum_i n_i \cdot \text{DBCV}_i}{\sum_i n_i}$$

where $n_i$ is the number of points in cluster $i$.

### Why Is DBCV Needed?

| Problem | How DBCV solves it |
|---------|--------------------|
| **Circular evaluation** — The baseline selects HDBSCAN parameters by maximising ARI against ground-truth labels. This is cheating: in a real application those labels don't exist. | DBCV is purely **internal** — it evaluates the clustering using only the data geometry, with no reference to external labels. |
| **Shape assumptions** — Silhouette Score and other common metrics assume convex, roughly spherical clusters. HDBSCAN intentionally discovers clusters of arbitrary shape. | DBCV is designed for **density-based** clusterings and correctly handles non-convex, irregularly shaped clusters. |
| **Noise awareness** — Standard metrics have no notion of noise points. | DBCV naturally integrates with HDBSCAN's noise model — only clustered points contribute to the score. |

### Interpretation

| DBCV Range | Meaning |
|------------|---------|
| **0.75 – 1.0** | Excellent — clusters are dense and well-separated |
| **0.50 – 0.75** | Good — clear density structure |
| **0.25 – 0.50** | Moderate — some cluster overlap |
| **0.00 – 0.25** | Weak — clusters barely distinguishable from surrounding space |
| **< 0.00** | Poor — clusters overlap significantly |

### Practical Note

DBCV requires the minimum spanning tree to be computed, which is why v2 sets `gen_min_span_tree=True` on every HDBSCAN call. Without this flag, accessing `clusterer.relative_validity_` raises an `AttributeError`.

### Reference

Moulavi, D., Jaskowiak, P.A., Campello, R.J.G.B., Zimek, A., & Sander, J. (2014). *Density-Based Clustering Validation*. In Proceedings of the 2014 SIAM International Conference on Data Mining (SDM), pp. 839–847.

---

## 1. Reproducibility — Random Seeds

**Problem:** The baseline sets `random_state=42` on individual UMAP objects but does not seed the global Python or NumPy random number generators. Any code path that draws from these (e.g. random sampling, pair generation) may produce different results across runs.

**Fix:** v2 adds a global `SEED` constant and seeds all relevant generators at the top of the notebook.

```python
# ── Baseline (cell-3) ──────────────────────────────────
import pandas as pd
import numpy as np
# ... no seeds set ...

# ── v2 (cell-3) ───────────────────────────────────────
import pandas as pd
import numpy as np
import random

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
```

## 2. Unused Import Removed

**Problem:** The baseline imports `from collections import Counter` but never uses it anywhere in the notebook.

**Fix:** v2 removes the import.

```python
# ── Baseline (cell-3) ──────────────────────────────────
from collections import Counter  # never used

# ── v2 (cell-3) ───────────────────────────────────────
# removed
```

## 3. UMAP — Precomputed Nearest-Neighbor Graph

**Problem:** The baseline creates two separate UMAP objects (15d for clustering, 2d for visualization) in separate cells. Each independently computes a nearest-neighbor graph over the same 26,872-point, 384-dimensional embedding matrix with the same `n_neighbors=15` and `metric="cosine"`. The NN search is the most expensive step in UMAP, so this work is done twice.

**Fix:** v2 computes the NN graph once using `umap.umap_.nearest_neighbors()` and passes the result tuple `(knn_indices, knn_dists, knn_forest)` to both UMAP instances via the `precomputed_knn` parameter.

```python
# ── Baseline (cell-11 and cell-12) ─────────────────────
# Cell 11: UMAP clustering — builds its own NN graph
umap_clustering = umap.UMAP(
    n_components=15, n_neighbors=15, min_dist=0.0,
    metric="cosine", random_state=42, verbose=True
)
embeddings_umap_cluster = umap_clustering.fit_transform(embeddings)

# Cell 12: UMAP viz — builds its own NN graph (again)
umap_viz = umap.UMAP(
    n_components=2, n_neighbors=15, min_dist=0.1,
    metric="cosine", random_state=42, verbose=True
)
embeddings_2d = umap_viz.fit_transform(embeddings)

# ── v2 (cell-11 and cell-12) ──────────────────────────
# Cell 11: compute NN graph once
from umap.umap_ import nearest_neighbors
knn_indices, knn_dists, knn_forest = nearest_neighbors(
    embeddings, n_neighbors=15, metric="cosine", metric_kwds={},
    angular=True, random_state=np.random.RandomState(SEED), verbose=True
)
precomputed_knn = (knn_indices, knn_dists, knn_forest)

# Cell 12: both UMAP projections reuse the precomputed NN graph
umap_clustering = umap.UMAP(
    ..., precomputed_knn=precomputed_knn
)
umap_viz = umap.UMAP(
    ..., precomputed_knn=precomputed_knn
)
```

**Impact:** Eliminates one full NN search (~10-15 seconds on this dataset), roughly halving the UMAP wall time.

## 4. Reusable Scatter Plot Helper

**Problem:** The baseline has nearly identical scatter-plotting code duplicated across multiple cells (ground-truth category, ground-truth intent, HDBSCAN clusters, side-by-side comparison, annotated plot). Each copy handles noise, palettes, and axis labels inline.

**Fix:** v2 extracts a `umap_scatter()` function that encapsulates the common logic:

```python
def umap_scatter(df, color_col, ax, palette, noise_labels=None, s=3, alpha=0.3):
    """Reusable UMAP scatter plot colored by a categorical column."""
    labels_sorted = sorted(df[color_col].unique())
    if noise_labels is not None:
        noise_mask = df[color_col].isin(...)
        if noise_mask.any():
            ax.scatter(..., c="lightgrey", s=1, alpha=0.2, label=f"Noise (...)")
        labels_sorted = [l for l in labels_sorted if l not in ...]
    for i, label in enumerate(labels_sorted):
        mask = df[color_col] == label
        ax.scatter(...)
    ax.set_xlabel("UMAP-1")
    ax.set_ylabel("UMAP-2")
```

All 5 scatter plot cells now call `umap_scatter()` with different parameters, eliminating ~60 lines of duplicated code.

## 5. Expanded HDBSCAN Parameter Sweep

**Problem:** The baseline sweeps only over `min_cluster_size` in `[25, 50, 100, 200, 500]` with a fixed `min_samples=10` and no `cluster_selection_epsilon`. This is a 1D sweep with just 5 configurations.

**Fix:** v2 performs a 3D sweep over three parameters:

| Parameter | Baseline | v2 |
|-----------|----------|----|
| `min_cluster_size` | `[25, 50, 100, 200, 500]` | `[200, 500, 750, 1000]` |
| `min_samples` | fixed at `10` | `[5, 10, 25, 50]` |
| `cluster_selection_epsilon` | not used (`0.0`) | `[0.0, 0.5, 1.0, 2.0]` |
| **Total configs** | **5** | **64** |

The `cluster_selection_epsilon` parameter is particularly important — it merges HDBSCAN leaf clusters that are within `epsilon` distance of each other, directly controlling the granularity of the final partition. Larger `min_cluster_size` values also shift the sweep toward coarser clusterings that are more aligned with the 27 ground-truth intents.

## 6. Selection Metric — DBCV Instead of ARI

**Problem:** The baseline selects the best HDBSCAN configuration by maximising ARI (Adjusted Rand Index) against the ground-truth intent labels. This is **circular** — the unsupervised clustering is being tuned using the very labels it aims to discover. In a real-world scenario these labels would not be available.

**Fix:** v2 selects by **DBCV** (Density-Based Clustering Validation), an internal validity metric available via `clusterer.relative_validity_`. DBCV measures how well the clustering captures the density structure of the data without reference to any external labels.

```python
# ── Baseline (cell-18) ─────────────────────────────────
best_mcs = sweep_df.loc[sweep_df["ari_vs_intent"].idxmax(), "min_cluster_size"]

# ── v2 (cell-18) ──────────────────────────────────────
feasible = sweep_df[sweep_df["n_clusters"] <= 30]
best_row = feasible.loc[feasible["dbcv"].idxmax()]
```

ARI is still computed and displayed for **comparison**, but it no longer drives parameter selection.

## 7. Cluster Count Constraint (<= 30)

**Problem:** Optimising DBCV without constraints can favour configurations with many tiny, tightly-packed clusters. DBCV rewards high intra-cluster density and low inter-cluster density, which is maximised when each dense region gets its own cluster — even if that means hundreds of micro-clusters.

**Fix:** v2 filters the sweep results to only configurations producing **<= 30 clusters** before selecting the best DBCV score. This ensures the result is practically useful (comparable to the 27 ground-truth intents) while still using an unsupervised metric within that feasible region.

```python
feasible = sweep_df[sweep_df["n_clusters"] <= 30]
if feasible.empty:
    raise RuntimeError("No configurations produced <= 30 clusters — expand the sweep grid.")
best_row = feasible.loc[feasible["dbcv"].idxmax()]
```

The threshold of 30 was chosen to provide a small margin above the 27 known intents, allowing for the possibility that HDBSCAN discovers legitimate sub-intents.

## 8. DBCV Computation Enabled

**Problem:** HDBSCAN's `relative_validity_` attribute (DBCV score) requires the minimum spanning tree to be computed, which is disabled by default. The baseline does not set `gen_min_span_tree=True`, so accessing `relative_validity_` would raise an `AttributeError`.

**Fix:** v2 adds `gen_min_span_tree=True` to every HDBSCAN constructor — both in the sweep loop and in the final clustering run.

```python
c = hdbscan.HDBSCAN(
    min_cluster_size=mcs,
    min_samples=ms,
    metric="euclidean",
    cluster_selection_method="eom",
    cluster_selection_epsilon=eps,
    gen_min_span_tree=True   # <-- enables DBCV
)
```

## 9. Full-Dataset Evaluation via Approximate Predict

**Problem:** The baseline evaluates ARI and NMI only on non-noise points (those with `hdbscan_cluster != -1`). When 18.7% of the data is labelled as noise and excluded, the reported metrics are optimistic — the hardest-to-cluster points are simply dropped.

**Fix:** v2 uses `hdbscan.approximate_predict()` to assign noise points to their nearest cluster, then reports metrics in **two modes**:

| Scope | Description |
|-------|-------------|
| **Excl. noise** | Standard approach (matches baseline), evaluates only assigned points |
| **Full dataset** | All 26,872 points evaluated after noise assignment via `approximate_predict` |

```python
# ── Baseline (cell-23) ─────────────────────────────────
valid_mask = df["hdbscan_cluster"] != -1
ari_intent = adjusted_rand_score(df.loc[valid_mask, "intent_label"], pred)
# (only non-noise points evaluated)

# ── v2 (cell-23) ──────────────────────────────────────
# Excl. noise (same as baseline)
ari_intent = adjusted_rand_score(df.loc[valid_mask, "intent_label"], pred)

# Full dataset: assign noise to nearest cluster
labels_full = cluster_labels.copy()
noise_idx = np.where(labels_full == -1)[0]
if len(noise_idx) > 0:
    approx_labels, _ = hdbscan.approximate_predict(clusterer, embeddings_umap_cluster[noise_idx])
    labels_full[noise_idx] = approx_labels
ari_intent_full = adjusted_rand_score(df["intent_label"], labels_full)
```

This gives a more honest picture of how well the clustering covers the entire dataset.

## 10. Cosine Distance for Representative Sampling

**Problem:** When selecting representative utterances for LLM-based labeling, the baseline picks the samples closest to the cluster centroid using **Euclidean distance**. However, the entire pipeline (UMAP, sentence embeddings) operates in cosine space. Euclidean distance in high dimensions can be dominated by vector magnitude rather than direction.

**Fix:** v2 uses **cosine distance** (1 - cosine similarity) for centroid-based sampling, which is consistent with the rest of the pipeline.

```python
# ── Baseline (cell-32) ─────────────────────────────────
def get_representative_samples(df, cluster_id, embeddings, n=8):
    ...
    centroid = cluster_embs.mean(axis=0)
    distances = np.linalg.norm(cluster_embs - centroid, axis=1)  # Euclidean
    closest_idx = distances.argsort()[:n]
    ...

# ── v2 (cell-32) ──────────────────────────────────────
def get_representative_samples(df, cluster_id, embeddings, n=8):
    ...
    centroid = cluster_embs.mean(axis=0)
    norms_emb = np.linalg.norm(cluster_embs, axis=1)
    norm_cent = np.linalg.norm(centroid)
    cosine_dists = 1 - (cluster_embs @ centroid) / (norms_emb * norm_cent + 1e-10)  # cosine
    closest_idx = cosine_dists.argsort()[:n]
    ...
```

## 11. Concurrent LLM API Calls

**Problem:** The baseline labels clusters by calling the Anthropic API sequentially in a for-loop. With 21 clusters, this means 21 serial HTTP round-trips.

**Fix:** v2 uses `anthropic.AsyncAnthropic` with `asyncio.gather` to issue all LLM labeling requests concurrently.

```python
# ── Baseline (cell-32) ─────────────────────────────────
client = anthropic.Anthropic()
llm_labels = {}
for cid in sorted(valid_df["hdbscan_cluster"].unique()):
    response = client.messages.create(...)  # sequential
    llm_labels[cid] = response.content[0].text.strip()

# ── v2 (cell-32) ──────────────────────────────────────
async def label_cluster(async_client, cid, samples, tfidf_label):
    response = await async_client.messages.create(...)
    return cid, response.content[0].text.strip()

async def label_all_clusters():
    async_client = anthropic.AsyncAnthropic()
    tasks = [label_cluster(async_client, cid, ...) for cid in cluster_ids]
    results = await asyncio.gather(*tasks)  # concurrent
    return dict(results)

llm_labels = await label_all_clusters()
```

**Impact:** With 21 clusters, the total LLM labeling time drops from ~21x a single call to roughly 1-2x (bounded by the slowest response), a ~10-15x speedup.

---

## Baseline Results (from `clustering_analysis.ipynb`)

| Setting | Value |
|---------|-------|
| Sweep type | 1D (`min_cluster_size` only, 5 configs) |
| Selection metric | ARI vs intent (circular) |
| Best `min_cluster_size` | 500 |
| `min_samples` | 10 (fixed) |
| `cluster_selection_epsilon` | 0.0 (not used) |
| Clusters found | 21 |
| Noise points | 5,022 (18.7%) |
| DBCV | not computed |

| Metric | Value | Scope |
|--------|-------|-------|
| ARI vs Intent (27) | 0.6961 | Excl. noise only |
| NMI vs Intent (27) | 0.8751 | Excl. noise only |
| ARI vs Category (11) | 0.6317 | Excl. noise only |
| NMI vs Category (11) | 0.8368 | Excl. noise only |
| Mean per-intent purity | 0.883 | Excl. noise only |
| Min per-intent purity | 0.446 | Excl. noise only |

---

## How to Compare Results

After running both notebooks end-to-end on the same dataset, compare:

1. **Cluster count** — both should produce a manageable number of clusters (< 30)
2. **Noise %** — v2 may have different noise rates depending on the DBCV-optimal parameters
3. **ARI / NMI (excl. noise)** — direct comparison with the baseline's reported metrics
4. **ARI / NMI (full dataset)** — v2-only; gives an honest evaluation including the hardest points
5. **DBCV** — v2-only; internal validity measure (higher is better)
6. **Per-intent purity** — how cleanly each ground-truth intent maps to a single cluster

The v2 results should be compared with caution: since v2 selects parameters by DBCV rather than ARI, its ARI may be slightly lower than the baseline's (which was optimised directly for ARI). The more meaningful comparison is whether the clustering is *structurally sound* (high DBCV, reasonable cluster count) while still achieving competitive ARI/NMI.